In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import types as T

In [2]:
# In[2]:


spark = SparkSession.builder.appName('abc')\
.config('spark.driver.memory' , '100g')\
.config('spark.sql.legacy.timeParserPolicy' , 'LEGACY')\
.config("spark.sql.shuffle.partitions" , "500")\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 07:51:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/09 07:51:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# In[4]:


raw_data_path = '/home/data/manu/sample/cibil/'
prep_data_path = '/home/jupyter-monica.marmit/feature_enginerring/cibil/output/'

In [4]:
# In[3]:


run_date = '2024-05-01'

In [5]:
# In[5]:


prods = ['PL' , 'GL' , 'CV' , 'TW' , 'BL' , 'HL' , 'Other' , 'OD' , 'AL' , 'CC' , 'LAP' , 'CE' , 'EL' , 'CD']
prods0 = ['PL' , 'GL' , 'TW' , 'BL' , 'HL' , 'CD' , 'AL']
prods1 = [p + "_" for p in prods0]

In [6]:
# In[ ]:


# partitioning the record on id

w1 = Window.partitionBy('MemberReference').orderBy('DATEOPENED' , F.desc('SANCTIONAMOUNT_new') , 'PRODUCT_DESC')
w2 = Window.partitionBy('MemberReference').orderBy(F.desc('DATEOPENED') , F.desc('SANCTIONAMOUNT_new') , 'PRODUCT_DESC')
w3 = Window.partitionBy('MemberReference' , 'PRODUCT_DESC').orderBy(F.desc('DATEOPENED') , F.desc('SANCTIONAMOUNT_new'))

In [7]:
# In[ ]:


leverage_exp1 = ["*"]\
+[
    f"""
    case when mn_diff_retro_open < {i} or  
    (
        ( DATECLOSED is not null and mn_diff_retro_close > {i} ) or
        ( 
            BALANCE = 0 and
            ACCOUNT_TYPE not in (
                'Corporate Credit Card' , 'Credit Card' , 'Kisan Credit Card' , 'Secured Credit Card' ,
                'Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft' , 
                'Non - Funded Credit Facility' ,
                'Business Non - Funded Credit Facility - General' , 
                'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
                'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
                'Business Non - Funded Credit Facility - Priority Sector - Others' 
            ) and
            ( LASTPAYMENTDATE is not null and mn_diff_retro_payment > {i} )
        )
    ) then 0
    else 1 end as is_open_m{i}
    """ for i in range(0 , 13)
]\
+[
    """
    case when ACCOUNT_TYPE in (
        'Corporate Credit Card' , 'Credit Card' , 'Kisan Credit Card' , 'Secured Credit Card' ,
        'Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft' ,
        'Non - Funded Credit Facility' ,
        'Business Non - Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' 
    ) then 1 else 0 end as cc_like_acct_list
    """
]\
+["case when SANCTIONAMOUNT_NEW - BALANCE >= 0 then SANCTIONAMOUNT_NEW else BALANCE end as SANCTIONAMOUNT_EMI"]\
+[f"case when mn_diff_retro_open <= {i} then 1 else 0 end as loan_vintage_{i}_mn" for i in [1 , 3 , 6 , 12 , 24 , 48]]\
+["1 as loan_vintage_all_mn"]

leverage_exp2 = ["*"]\
+[
    """
    case when mn_diff_retro_end = 0 then 1 
    when mn_diff_retro_end <= 36 then cnt_0
    else cnt_0 + mn_diff_retro_end - 36 end as count_dpd_0
    """
]\
+["case when (SANCTIONAMOUNT_EMI <= 0) or (SANCTIONAMOUNT_EMI is null) then 0 else SANCTIONAMOUNT_EMI end as SANCTIONAMOUNT_EMI1"]

leverage_exp3 = ["*"]\
+[
    """
    case when (cc_like_acct_list = 0 and SANCTIONAMOUNT_EMI1 > 0 and SANCTIONAMOUNT_EMI1 <= 5000) then SANCTIONAMOUNT_EMI1
    when (
        cc_like_acct_list = 0 and SANCTIONAMOUNT_EMI1 > 5000 and SANCTIONAMOUNT_EMI1 > BALANCE and count_dpd_0 > 0
    ) then (SANCTIONAMOUNT_EMI1 - BALANCE) / count_dpd_0
    when PRODUCT_DESC in ('CC' , 'PL') and SANCTIONAMOUNT_EMI1 > 0 then 0.05 * SANCTIONAMOUNT_EMI1
    when PRODUCT_DESC in ('HL' , 'LAP') and SANCTIONAMOUNT_EMI1 > 0 then 0.007 * SANCTIONAMOUNT_EMI1
    when SANCTIONAMOUNT_EMI1 > 0 then 0.03 * SANCTIONAMOUNT_EMI1 
    else 0 end as EMI_new
    """
]

leverage_exp4 = ["*"]\
+[
    f"""
    case when is_open_m{i} = 1 then EMI_new 
    else 0 end as EMI_M{i}""" 
    for i in range(0 , 13)
]\
+["case when SANCTIONAMOUNT_EMI1 > 30000 then 1 else 0 end as long_term_flag"]\
+["case when SANCTIONAMOUNT_EMI1 <= 30000 then 1 else 0 end as short_term_flag"]

leverage_exp5 = ["*"]\
+[f"case when long_term_flag = 1 then EMI_M{i} else 0 end as lt_emi_m{i}" for i in range(0 , 13)]\
+[f"case when short_term_flag = 1 then EMI_M{i} else 0 end as st_emi_m{i}" for i in range(0 , 13)]\
+[f"case when secured = 1 then EMI_M{i} else 0 end as secure_emi_m{i}" for i in range(0 , 13)]\
+[f"case when unsecured = 1 then EMI_M{i} else 0 end as unsecure_emi_m{i}" for i in range(0 , 13)]\
+[f"case when installment = 1 then EMI_M{i} else 0 end as install_emi_m{i}" for i in range(0 , 13)]\
+[f"case when revolving = 1 then EMI_M{i} else 0 end as revolve_emi_m{i}" for i in range(0 , 13)]\
+[
    f"case when PRODUCT_DESC = '{prod}' then EMI_M{i} else 0 end as {prod}_emi_m{i}" 
    for i in range(0 , 13) 
    for prod in ['AL' , 'BL' , 'PL' , 'HL' , 'CC' , 'CD']
]\
+[
    f"case when PRODUCT_DESC = 'PL' and {flag} = 1 then EMI_M{i} else 0 end as PL_{flag.split('_')[0]}_emi_m{i}" 
    for i in range(0 , 13)
    for flag in ['long_term_flag' , 'short_term_flag']
]

leverage_exp6 = ['*']\
+[
    f"{type}emi_m0 - {type}emi_{month} as diff_{type}emi_m0_{month}" 
    for type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' ,  'PL_short_' , 'PL_long_' 
    ]
    for month in ['m1' , 'm3' , 'm6' , 'm12']
]\
+ [
    " + ".join([f"{type}emi_m{month}" for month in range(0,3)]) + f" as sum_{type}emi_m0_m2"
    for type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
]\
+ [
    " + ".join([f"{type}emi_m{month}" for month in range(0,6)]) + f" as sum_{type}emi_m0_m5"
    for type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
]\
+ [
    " + ".join([f"{type}emi_m{month}" for month in range(0,12)]) + f" as sum_{type}emi_m0_m11"
    for type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
]

leverage_agg_exp1 = []\
+ [
    F.expr(f"{agg_func}({emi_type}emi_{month} * {vintage_flag}) as {agg_func}_{emi_type}emi_{month}_vintage_{vintage_flag.split('_')[2]}_mn")
    for agg_func in ['sum' , 'max']
    for emi_type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
    for month in ['m0' , 'm1' , 'm2' , 'm3' , 'm4' , 'm5' , 'm6' , 'm7' , 'm8' , 'm9' , 'm10' , 'm11' , 'm12'] 
    for vintage_flag in [
        'loan_vintage_1_mn' , 'loan_vintage_6_mn' , 'loan_vintage_12_mn' , 
        'loan_vintage_24_mn' , 'loan_vintage_48_mn' , 'loan_vintage_all_mn'
    ]
    if (month == 'm0') or ((emi_type in ('' , 'lt_' , 'st_' , 'secure_')) and (agg_func == 'sum') and (vintage_flag == 'loan_vintage_all_mn')) 
]\
+ [
    F.expr(f"{agg_func}({emi_type}emi_{month} * {vintage_flag}) as {agg_func}_{emi_type}emi_{month}_vintage_{vintage_flag.split('_')[2]}_mn")
    for agg_func in ['sum']
    for emi_type in [
        'diff_' , 'diff_lt_' , 'diff_st_' , 'diff_secure_' , 'diff_unsecure_' , 'diff_install_' , 'diff_revolve_' , 'diff_CD_' ,
        'diff_AL_' , 'diff_BL_' , 'diff_PL_' , 'diff_HL_' , 'diff_CC_' , 'diff_PL_short_' , 'diff_PL_long_' 
    ]
    for month in ['m0_m3' , 'm0_m6' , 'm0_m12'] 
    for vintage_flag in ['loan_vintage_24_mn' , 'loan_vintage_48_mn' , 'loan_vintage_all_mn']
]\
+ [
    F.expr(f"sum({agg_func}{emi_type}emi_{month} * {vintage_flag}) as sum_{agg_func}{emi_type}emi_{month}_vintage_{vintage_flag.split('_')[2]}_mn")
    for agg_func in ['sum_']
    for emi_type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
    for month in ['m0_m2' , 'm0_m5' , 'm0_m11']
    for vintage_flag in ['loan_vintage_24_mn' , 'loan_vintage_48_mn' , 'loan_vintage_all_mn']
]

leverage_agg_exp2 = ["*"]\
+ [
    f"""
    case when sum_{type}emi_{month}_{vintage_flag} is null then 0
    when (sum_diff_emi_{month}_{vintage_flag} is null) or (sum_diff_emi_{month}_{vintage_flag} = 0) then sum_{type}emi_{month}_{vintage_flag}
    else sum_{type}emi_{month}_{vintage_flag} / sum_diff_emi_{month}_{vintage_flag} end as ratio_{type}emi_diff_total_emi_{month}_{vintage_flag}
    """
    for type in [
        'diff_lt_' , 'diff_st_' , 'diff_secure_' , 'diff_unsecure_' , 'diff_install_' , 'diff_revolve_' , 'diff_CD_' ,
        'diff_AL_' , 'diff_BL_' , 'diff_PL_' , 'diff_HL_' , 'diff_CC_' , 'diff_PL_short_' , 'diff_PL_long_' 
    ]
    for month in ['m0_m3' , 'm0_m6' , 'm0_m12']
    for vintage_flag in ['vintage_24_mn' , 'vintage_48_mn' , 'vintage_all_mn']
    
]\
+ [
    f"""
    case when sum_sum_{type}emi_{month}_{vintage_flag} is null then -1
    when (sum_sum_emi_{month}_{vintage_flag} is null) then -2 
    when (sum_sum_emi_{month}_{vintage_flag} = 0) then -3 
    else sum_sum_{type}emi_{month}_{vintage_flag} / sum_sum_emi_{month}_{vintage_flag} end as ratio_sum_{type}emi_sum_total_emi_{month}_{vintage_flag}
    """
    for type in [
        'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
    for month in ['m0_m2' , 'm0_m5' , 'm0_m11']
    for vintage_flag in ['vintage_24_mn' , 'vintage_48_mn' , 'vintage_all_mn']
]\
+[
    "max_emi_m0_m12 - sum_emi_m0_vintage_all_mn as residual_emi_m0"
]\
+[
    "max_emi_m0_m12 - sum_emi_m1_vintage_all_mn as residual_emi_m1"
]

leverage_agg_exp3 = ["*"]\
+ [
    " + ".join([f"sum_{loan_type}emi_m{month}_vintage_all_mn / 3" for month in range(0,3)]) + f" as avg_{loan_type}emi_m0_m2" 
    for loan_type in ('' , 'lt_' , 'st_' , 'secure_')
]\
+ [
    " + ".join([f"sum_{loan_type}emi_m{month}_vintage_all_mn / 3" for month in range(6,9)]) + f" as avg_{loan_type}emi_m6_m8" 
    for loan_type in ('' , 'lt_' , 'st_' , 'secure_')
]\
+ [
    f"avg_{loan_type}emi_m0_m2 - avg_{loan_type}emi_m6_m8 as diff_avg_{loan_type}emi_m0_m2_and_m6_m8"
    for loan_type in ('' , 'lt_' , 'st_' , 'secure_')
]\
+ [
    f"""
    case when avg_{loan_type}emi_m6_m8 = 0 then 0 
    else avg_{loan_type}emi_m0_m2 / avg_{loan_type}emi_m6_m8
    end as ratio_avg_{loan_type}emi_m0_m2_and_m6_m8
    """
    for loan_type in ('' , 'lt_' , 'st_' , 'secure_')
]

In [8]:

tenure_exp1 = ["*"]\
+ [
    """
    case when is_open_m0 = 1 then null
    when DATECLOSED is not null then DATECLOSED
    when (LASTPAYMENTDATE > date_trunc('month' , DATEOPENED)) then LASTPAYMENTDATE 
    else PAYMENT_HISTORY_START_DATE end as ACCOUNT_CLOSING_DATE
    """
]

tenure_exp2 = ["*"]\
+ [
    f"""
    case when loan_tenure <= {i} then loan_tenure end as loan_tenure_{i}_mn 
    """
    for i in [1 , 6 , 12 , 24 , 48]
]\
+ [
    f"""
    case when {flag} = 1 then loan_tenure end as {flag.split('_')[0]}_loan_tenure
    """
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+ [
    f"""
    case when {flag} = 1 and loan_tenure <= {i} then loan_tenure end as {flag.split('_')[0]}_loan_tenure_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+[
    f"case when PRODUCT_DESC = '{prod}' then loan_tenure end as {prod}_loan_tenure"
    for prod in ['AL' , 'BL' , 'PL' , 'HL' , 'CC' , 'CD']
]\
+[
    f"case when PRODUCT_DESC = '{prod}' and loan_tenure <= {i} then loan_tenure end as {prod}_loan_tenure_{i}_mn"
    for prod in ['AL' , 'BL' , 'PL' , 'HL' , 'CC' , 'CD']
    for i in [1 , 6 , 12 , 24 , 48]
]\
+ [
    f"""
    {agg}(SANCTIONAMOUNT_NEW * int(loan_tenure <= {i}) * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC) as {prod}_{agg}_Saction_Amt_tenure_{i}m 
    """
    for agg in ['max' , 'min']
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+ [
    f"""
    {agg}(SANCTIONAMOUNT_NEW * loan_vintage_{i}_mn * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC) as {prod}_{agg}_Saction_Amt_vintage_{i}m 
    """
    for agg in ['max' , 'min']
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+ [
    f"""
    {agg}(BALANCE * int(loan_tenure <= {i}) * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC) as {prod}_{agg}_balance_tenure_{i}m 
    """
    for agg in ['max' , 'sum']
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+ [
    f"""
    {agg}(BALANCE * loan_vintage_{i}_mn * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC) as {prod}_{agg}_balance_vintage_{i}m 
    """
    for agg in ['max' , 'sum']
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+ [
    f"""
    (BALANCE * int(row_num = 1) * int(PRODUCT_DESC = '{prod}')) as latest_{prod}_balance 
    """
    for prod in prods0
]\
+ [
    f"""
    (SANCTIONAMOUNT_NEW * int(row_num = 1) * int(PRODUCT_DESC = '{prod}')) as latest_{prod}_sanction_amt
    """
    for prod in prods0
]\
+ [
    f"""
    (is_open_m0 * int(row_num = 1) * int(PRODUCT_DESC = '{prod}')) as latest_{prod}_current_status
    """
    for prod in prods0
]\
+ [
    f"""
    (mn_diff_retro_open * int(row_num = 1) * int(PRODUCT_DESC = '{prod}')) as latest_{prod}_vintage
    """
    for prod in prods0
]


tenure_exp3 = ["*"]\
+[
    f"case when is_open_m0 = 1 then {type}loan_tenure{duration} end as live_{type}loan_tenure{duration}"
    for type in [
        '' , 'long_' , 'short_' , 'secured_' , 'unsecured_' , 'installment_' , 'revolving_' , 
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'CD_'
    ]
    for duration in ['' , '_1_mn' , '_6_mn' , '_12_mn' , '_24_mn' , '_48_mn']
]

tenure_agg_exp1 = []\
+[
    F.expr(
        f"""
        {agg_func}({status}{type}loan_tenure{duration} * {vintage_flag}) 
        as {agg_func}_{status}{type}loan_tenure{duration}_vintage_{vintage_flag.split('_')[2]}_mn
        """
    )
    for agg_func in ['count' , 'max' , 'avg']
    for status in ['' , 'live_']
    for type in [
        '' , 'long_' , 'short_' , 'secured_' , 'unsecured_' , 'installment_' , 'revolving_' , 
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'CD_'
    ]
    for duration in ['' , '_1_mn' , '_6_mn' , '_12_mn' , '_24_mn' , '_48_mn']
    for vintage_flag in [
        'loan_vintage_1_mn' , 'loan_vintage_6_mn' , 'loan_vintage_12_mn' , 
        'loan_vintage_24_mn' , 'loan_vintage_48_mn' , 'loan_vintage_all_mn'
    ]
    if (vintage_flag == 'loan_vintage_all_mn') or 
    ((duration != '') and (status == '') and (agg_func == 'count') and (int(vintage_flag.split('_')[2]) > int(duration.split('_')[1]))) 
]\
+ [
    F.expr(
        f"""
        {agg_func}({prod}_{agg_func}_Saction_Amt_{duration_type}_{i}m) as {prod}_{agg_func}_Saction_Amt_{duration_type}_{i}m 
        """
    )
    for agg_func in ['min' , 'max']
    for prod in prods0
    for i in [1 , 6 , 12 , 24 , 48]
    for duration_type in ['tenure' , 'vintage']
]\
+ [
    F.expr(
        f"""
        {agg_func}({prod}_{agg_func}_balance_{duration_type}_{i}m) as {prod}_{agg_func}_balance_{duration_type}_{i}m 
        """
    )
    for agg_func in ['sum' , 'max']
    for prod in prods0
    for i in [1 , 6 , 12 , 24 , 48]
    for duration_type in ['tenure' , 'vintage']
]\
+ [
    F.max(F.col(f"latest_{prod}_{data_type}")).alias(f"latest_{prod}_{data_type}")
    for prod in prods0
    for data_type in ['balance' , 'sanction_amt' , 'current_status' , 'vintage']
]

In [9]:
loan_status_exp1 = ["*"]\
+[
    f"""
    case when mn_retro_acct_close is not null and mn_retro_acct_close <= {i} then 1 else null end as num_closed_accts_last_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
]\
+[
    f"""
    case when mn_retro_acct_close is not null and mn_retro_acct_close <= {i} and {flag} = 1 then 1 
    else null end as num_closed_{flag.split('_')[0]}_accts_last_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+[
    f"""
    case when mn_retro_acct_close is not null and mn_retro_acct_close <= {i} and PRODUCT_DESC = '{prod}' then 1 
    else null end as num_closed_{prod}_accts_last_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+[
    f"""
    case when mn_retro_acct_close is not null and mn_retro_acct_close <= {i} and PRODUCT_DESC = 'PL' and {flag} = 1 then 1 
    else null end as num_closed_PL_{flag.split('_')[0]}_accts_last_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag']
]

loan_status_exp2 = ["*"]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} then 1 else null end as num_open_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and {flag} = 1 then 1 
    else null end as num_open_{flag.split('_')[0]}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and PRODUCT_DESC = '{prod}' then 1 
    else null end as num_open_{prod}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and PRODUCT_DESC = 'PL' and {flag} = 1 then 1 
    else null end as num_open_PL_{flag.split('_')[0]}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag']
]

loan_status_exp3 = ["*"]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and is_open_m0 = 1 then 1 else null end as num_live_open_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and {flag} = 1 and is_open_m0 = 1 then 1 
    else null end as num_live_open_{flag.split('_')[0]}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and PRODUCT_DESC = '{prod}' and is_open_m0 = 1 then 1 
    else null end as num_live_open_{prod}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for prod in prods0
]\
+[
    f"""
    case when mn_diff_retro_open is not null and mn_diff_retro_open <= {i} and PRODUCT_DESC = 'PL' and is_open_m0 = 1 and {flag} = 1 then 1 
    else null end as num_live_open_PL_{flag.split('_')[0]}_accts_vintage_{i}_mn
    """
    for i in [1 , 6 , 12 , 24 , 48]
    for flag in ['long_term_flag' , 'short_term_flag']
]

loan_status_agg_exp1 = []\
+[
    F.expr(f"sum(num_{cur_status}open_{type}accts_vintage_{i}_mn) as total_num_{cur_status}open_{type}accts_vintage_{i}_mn")
    for cur_status in ['' , 'live_']
    for type in [
        '' , 'long_' , 'short_' , 'secured_' , 'unsecured_' , 'installment_' , 'revolving_' ,
        'PL_short_' 
    ] + prods1
    for i in [1 , 6 , 12 , 24 , 48]
]\
+[
    F.expr(f"sum(num_closed_{type}accts_last_{i}_mn) as total_num_closed_{type}accts_last_{i}_mn")
    for type in [
        '' , 'long_' , 'short_' , 'secured_' , 'unsecured_' , 'installment_' , 'revolving_' ,
        'PL_short_'
    ] + prods1
    for i in [1 , 6 , 12 , 24 , 48]
]\
+[
    F.expr(f"sum(is_open_m{i}) as num_concurrent_live_loans_m{i}")
    for i in range(0 , 13)
]\
+[
    F.expr(f"sum(is_open_m{i} * {flag}) as num_{flag.split('_')[0]}_concurrent_live_loans_m{i}")
    for i in [0 , 1 , 6]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured' , 'installment' , 'revolving']
]\
+[
    F.expr(f"sum(is_open_m{i} * {flag} * int(PRODUCT_DESC = 'PL')) as num_{flag.split('_')[0]}_PL_concurrent_live_loans_m{i}")
    for i in [0 , 1 , 6]
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured']
]

loan_status_agg_exp2 = ["*"]\
+ [
    f"""
    case when (
        num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0 is null
    ) and (
        num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6 is null
    ) then 0
    when num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0 is null then -1*num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6
    when num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6 is null then num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0
    else num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0 - num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6
    end as diff_num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0_m6
    """ 
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured']
    for product_type in [''] + ['PL_']
]\
+ [
    f"""
    case when num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0 is null then 0
    when (
        num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6 is null
    ) or (
        num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6 = 0
    ) then num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0
    else num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0 / num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m6 
    end as ratio_num_{flag.split('_')[0]}_{product_type}concurrent_live_loans_m0_m6
    """
    for flag in ['long_term_flag' , 'short_term_flag' , 'secured' , 'unsecured']
    for product_type in [''] + ['PL_']
]\
+ [
    """
    case when (num_concurrent_live_loans_m0 is null) and (num_concurrent_live_loans_m6 is null) then 0
    when (num_concurrent_live_loans_m0 is null) then -1 * num_concurrent_live_loans_m6
    when (num_concurrent_live_loans_m6 is null) then num_concurrent_live_loans_m0
    else num_concurrent_live_loans_m0 - num_concurrent_live_loans_m6 
    end as diff_num_concurrent_live_loans_m0_m6
    """
]\
+ [
    """
    case when num_concurrent_live_loans_m0 is null then 0
    when (num_concurrent_live_loans_m6 is null) or (num_concurrent_live_loans_m6 = 0) then num_concurrent_live_loans_m0
    else num_concurrent_live_loans_m0 / num_concurrent_live_loans_m6 
    end as ratio_num_concurrent_live_loans_m0_m6
    """
]\
+[
    f"""
    case when num_concurrent_live_loans_m{i} = 0 then 0
    else sum_emi_m{i}_vintage_all_mn / num_concurrent_live_loans_m{i} 
    end as avg_emi_per_live_loans_m{i}_vintage_all_mn 
    """
    for i in range(0 , 13)
]\
+[
    "max_num_concurrent_live_loans_m0_m12 - num_concurrent_live_loans_m0 as diff_max_concurrent_live_loans_m0_m12_live_loans_m0"
]\
+[
    "max_num_concurrent_live_loans_m0_m12 - num_concurrent_live_loans_m1 as diff_max_concurrent_live_loans_m0_m12_live_loans_m1"
]

loan_status_agg_exp3 = ["*"]\
+[
    f"""
    avg_emi_per_live_loans_m0_vintage_all_mn - avg_emi_per_live_loans_m{i}_vintage_all_mn
    as diff_avg_emi_per_live_loans_m0_m{i}_vintage_all_mn
    """
    for i in [1 , 3 , 6 , 12]
]

In [10]:
tl_exp5=["*"]\
+[
    """
    case when ACCOUNT_TYPE not in ('Corporate Credit Card' , 'Credit Card' , 'Kisan Credit Card' , 'Secured Credit Card') then SANCTIONAMOUNT 
    when CREDITLIMIT is not null and CREDITLIMIT > 0 then CREDITLIMIT else greatest(HIGHCREDIT , SANCTIONAMOUNT) end as SANCTIONAMOUNT_new
    """
]\
+["months_between(date_trunc('month' , run_dt) , date_trunc('month' , DATEOPENED)) as tenure"]\
+["months_between(date_trunc('month' , run_dt) , date_trunc('month' , LASTPAYMENTDATE)) as msp"]\
+[
    """
    case when months_between(date_trunc('month' , run_dt) , date_trunc('month' , DATECLOSED)) >= 12 and 
        DATECLOSED is not null then 1 
    else 0 end as months_closed
    """
]\
+["months_between(date_trunc('month' , run_dt) , date_trunc('month' , PAYMENT_HISTORY_START_DATE)) as month_scrub_payhist_start"]\
+["1 as Trade"]\
+[
    """
    case when ACCOUNT_TYPE in (
        'Corporate Credit Card' , 'Credit Card' , 'Kisan Credit Card' , 'Secured Credit Card' ,
        'Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft' ,
        'Non - Funded Credit Facility' ,
        'Business Non - Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' 
    ) then 1 
    else 0 end as Revolving
    """
]\
+[
    """
    case when ACCOUNT_TYPE in (
        'Auto Loan (Personal)' , 'Housing Loan' , 'Property Loan' , 'Loan Against Shares/Securities' , 'Gold Loan' ,
        'Leasing' , 'Two-wheeler Loan' , 'Loan Against Bank Deposits' , 'Commercial Vehicle Loan' , 
        'GECL Loan Secured' , 'Secured Credit Card' , 'Used Car Loan' , 'Construction Equipment Loan' ,
        'Tractor Loan' , 'Microfinance Business Loan' , 'Microfinance Personal Loan' ,
        'Microfinance Housing Loan' , 'Microfinance Others' , 'P2P Auto Loan' , 
        'Pradhan Mantri Awas Yojana - Credit Link Subsidy Scheme MAY CLSS' , 'Business Loan - Secured' ,
        'Business Loan - General' , 'Business Loan - Priority Sector - Small Business' , 
        'Business Loan - Priority Sector - Agriculture' , 'Business Loan - Priority Sector - Others' ,
        'Business Non - Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' ,
        'Business Loan Against Bank Deposits' , 'Priority Sector - Gold Loan'
    ) then 1 
    else 0 end as SECURED
    """
]\
+[
    """
    case when (DATECLOSED is not null and DATECLOSED < date_trunc('month' , run_dt)) or
    (
        BALANCE = 0 and ACCOUNT_TYPE not in (
            'Corporate Credit Card' , 'Credit Card' , 'Kisan Credit Card' , 'Secured Credit Card' ,
            'Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft' ,
            'Non - Funded Credit Facility' ,
            'Business Non - Funded Credit Facility - General' , 
            'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
            'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
            'Business Non - Funded Credit Facility - Priority Sector - Others' 
        )
        and LASTPAYMENTDATE < date_trunc('month' , run_dt)
    ) then 0 
    else 1 end as Open
    """
 ]\
+["case when is_suit_filed_or_wilful_default = 1 or is_written_off_or_settled = 1 then 1 else 0 end as  WOF_Flag"]\
+[
    """
    case when ACCOUNT_TYPE in ('Tractor Loan' , 'Used Car Loan' , 'P2P Auto Loan' , 'Auto Loan (Personal)') then 'AL' 
    when ACCOUNT_TYPE in (
        'Loan On Credit Card' , 'Loan to Professional' , 'Microfinance Personal Loan' , 'Personal Loan' ,
        'Microfinance Others' , 'P2P Personal Loan'
    ) then 'PL' 
    when ACCOUNT_TYPE in ('Gold Loan') then 'GL' 
    when ACCOUNT_TYPE in (
        'Corporate Credit Card' , 'Credit Card' , 'Fleet Card' , 'Kisan Credit Card' , 
        'Loan Against Bank Deposits' , 'Secured Credit Card'
    ) then 'CC' 
    when ACCOUNT_TYPE in (
        'Business Loan - Secured' , 'Business Loan - General' , 'Business Loan - Priority Sector - Small Business' , 
        'Business Loan - Priority Sector - Agriculture' , 'Business Loan - Priority Sector - Others' ,
        'Business Non - Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' ,
        'Business Loan Against Bank Deposits' , 'Business Loan - Unsecured' , 
        'Mudra Loans - Shishu/Kishor/Tarun' , 'GECL Loan Unsecured' , 'GECL Loan Secured' , 'Microfinance Business Loan' 
    ) then 'BL'
    when ACCOUNT_TYPE in ('Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft') then 'OD'
    when ACCOUNT_TYPE in ('Consumer Loan') then 'CD'
    when ACCOUNT_TYPE in ('Housing Loan' , 'Microfinance Housing Loan' , 'Pradhan Mantri Awas Yojana - Credit Link Subsidy Scheme MAY CLSS') then 'HL'
    when ACCOUNT_TYPE in ('Education Loan' , 'P2P Education Loan') then 'EL'
    when ACCOUNT_TYPE in ('Construction Equipment Loan') then 'CE'
    when ACCOUNT_TYPE in ('Property Loan') then 'LAP'
    when ACCOUNT_TYPE in ('Loan Against Shares/Securities') then 'LAS'
    when ACCOUNT_TYPE in ('Two-wheeler Loan') then 'TW'
    when ACCOUNT_TYPE in ('Commercial Vehicle Loan') then 'CV'
    when ACCOUNT_TYPE in ('Leasing' , 'Non - Funded Credit Facility' , 'Other') then 'Other'
    else 'Other' end as PRODUCT_DESC
    """
]

In [11]:
tl_exp6=["*"]\
+["case when revolving = 0 then 1 else 0 end as installment"]\
+["case when secured = 0 then 1 else 0 end as unsecured"]\
+[
    """
    case when PRODUCT_DESC = 'BL' and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.03 * SANCTIONAMOUNT 
    when PRODUCT_DESC = 'CC' and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.05 * BALANCE 
    when PRODUCT_DESC = 'PL' and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.03 * SANCTIONAMOUNT 
    when PRODUCT_DESC = 'HL' and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.01 * SANCTIONAMOUNT 
    when PRODUCT_DESC = 'AL' and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.03 * SANCTIONAMOUNT 
    when PRODUCT_DESC not in ('BL' , 'CC' , 'PL' , 'HL' , 'AL') and BALANCE > 0 and SANCTIONAMOUNT > 0 and open = 1 then 0.03 * SANCTIONAMOUNT 
    else 0 end as EMI
    """
]

In [12]:
tl_exp_new0=["*"]\
+["case when mn_diff_retro_report < 4 then 1 else 0 end as mn_flag"]\
+["case when PRODUCT_DESC = 'CC' and dpd_bucket_1 >= 2 then 1 when PRODUCT_DESC != 'CC' and dpd_bucket_1 >= 1 then 1 else 0 end as cc_relax_flag"]

tl_exp_new = ["*"]\
+["case when open = 1 then PASTDUEAMOUNT else 0 end as open_overdue"]\
+["case when mn_flag = 1 then PASTDUEAMOUNT else 0 end as mn_overdue"]\
+["case when cc_relax_flag = 1 then PASTDUEAMOUNT else 0 end as cc_relax_dpd_overdue"]\
+["case when PRODUCT_DESC='CC' then PASTDUEAMOUNT else 0 end as cc_overdue"]\
+["case when PRODUCT_DESC!='CC' then PASTDUEAMOUNT else 0 end as non_cc_overdue"]

In [13]:
tl_exp7=["*"]\
+["case when open=1 and PRODUCT_DESC='"+prod+"' then 1 else 0 end as "+prod+"_ind" for prod in prods]\
+["case when open=1 and PRODUCT_DESC='"+prod+"' then BALANCE else null end as "+prod+"_bal" for prod in prods]\
+["case when open=1 and PRODUCT_DESC='"+prod+"' and tenure <=6 then SANCTIONAMOUNT else null end as "+prod+"_sanc_amt_6m" 
    for prod in prods if prod!='CC'
]\
+[
    """
    case when open=1 and PRODUCT_DESC='CC' and tenure <=6 then coalesce(CREDITLIMIT,greatest(SANCTIONAMOUNT,HIGHCREDIT)) 
    else null end as CC_sanc_amt_6m
    """
]\
+["case when open=1 and PRODUCT_DESC='"+prod+"' then EMI else null end as "+prod+"_emi" for prod in prods]\
+["case when PRODUCT_DESC='{prod}' then tenure else 0 end as {prod}_tenure".format(prod=prod) for prod in prods]\
+["case when Open=1  and tenure <=3 and PRODUCT_DESC='{prod}' then BALANCE end as {prod}_open_balance_3m".format(prod=prod) for prod in prods]\
+["case when Open=1  and tenure <=6 and PRODUCT_DESC='{prod}' then BALANCE end as {prod}_open_balance_6m".format(prod=prod) for prod in prods]\
+["case when tenure <= 1 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_num_open_last1m".format(prod=prod) for prod in prods]\
+["case when tenure <= 3 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_num_open_last3m".format(prod=prod) for prod in prods]\
+["case when tenure <= 6 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_num_open_last6m".format(prod=prod) for prod in prods]\
+["case when tenure <= 12 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_num_open_last12m".format(prod=prod) for prod in prods]\
+["case when Installment=1 and Open=1 then 1 else 0 end as Open_installment"]\
+["case when Installment=1 then BALANCE else 0 end as Install_balance"]\
+["case when Installment=1 and Open=1 then BALANCE else 0 end as Open_install_balance"]\
+["case when Installment=1 then EMI else 0 end as Monthly_payment_amt_install"]\
+["case when Installment=1 and Open=1 then EMI else 0 end as Monthly_payment_amt_open_install"]\
+["case when Installment=1 and Open=1 then SANCTIONAMOUNT else 0 end as Loan_amt_open_install"]\
+["case when Open=1 and Revolving=1 then 1 else 0 end as Open_revolving"]\
+["case when Revolving=1 then BALANCE else 0 end as Revolving_balance_amt"]\
+["case when Revolving=1 and Open=1 then BALANCE else 0 end as Open_Revolving_balance_amt"]\
+["case when Revolving=1 then EMI else 0 end as Monthly_payment_revolving"]\
+["case when Revolving=1 then coalesce(CREDITLIMIT,greatest(SANCTIONAMOUNT,HIGHCREDIT)) else 0 end as Revolving_credit_limit"]\
+["case when Revolving=1 and open=1 then coalesce(CREDITLIMIT,greatest(SANCTIONAMOUNT,HIGHCREDIT)) else 0 end as Revolving_open_credit_limit"]\
+["case when SECURED=1 and open=1 then EMI else 0 end as secured_open_emi"]\
+["case when UNSECURED=1 and open=1 then EMI else 0 end as unsecured_open_emi"]\
+["case when Open=1 then BALANCE else 0 end as open_balance"]\
+["case when Open=1 then EMI else 0 end as Monthly_payment_open"]\
+["case when SECURED=1 then HIGHCREDIT else 0 end as secured_high_credit_amt"]\
+["case when UNSECURED=1 then HIGHCREDIT else 0 end as unsecured_high_credit_amt"]\
+["case when installment =1 then SANCTIONAMOUNT end as LOAN_AMOUNT"]\
+["case when SECURED =1 then tenure end as sec_tenure"]\
+["case when dpd_bucket_1 is null then null when dpd_bucket_1 in (-2,0,1) then 1 else 0 end as curr_satisfactory"]\
+["case when SECURED=1 then BALANCE end as secured_balance"]\
+["case when UNSECURED=1 then BALANCE end as unsecured_balance"]\
+["case when Open=1  and tenure <=3 then BALANCE end as open_balance_3m"]\
+["case when Open=1  and tenure <=6 then BALANCE end as open_balance_6m"]\
+["case when tenure <= 1 then 1 else null end as num_open_last1m"]\
+["case when tenure <= 3 then 1 else null end as num_open_last3m"]\
+["case when tenure <= 6 then 1 else null end as num_open_last6m"]\
+["case when tenure <= 12 then 1 else null end as num_open_last12m"]\
+["case when Open = 1 and Installment=1 then tenure else null end as tenure_open_install"]\
+["case when Open=1 and Revolving=1 then tenure else null end as tenure_open_revol"]\
+["case when Revolving=1 then tenure else null end as tenure_all_revol"]\
+["case when Installment=1 then tenure else null end as tenure_all_install"]\
+[
    "case when wof_flag=1 then 0 when ( "+" and ".join(["dpd_bucket_"+str(i)+" in (-2,0,1)" for i in range(1,25)])+" ) then 1 \
    when ( "+" or ".join(["dpd_bucket_"+str(i)+" >= 2" for i in range(1,25)])+" ) then 0 \
    else null end as satisfactory_flag_24m"
]\
+[
    "case when wof_flag=1 then 0 when ( "+" and ".join(["dpd_bucket_"+str(i)+" in (-2,0,1)" for i in range(1,13)])+" ) then 1 \
    when ( "+" or ".join(["dpd_bucket_"+str(i)+" >= 2" for i in range(1,13)])+" ) then 0 \
    else null end as satisfactory_flag_12m"
]\
+[
    "case when wof_flag=1 then 0 when ( "+" and ".join(["dpd_bucket_"+str(i)+" in (-2,0,1)" for i in range(1,7)])+" ) then 1 \
    when ( "+" or ".join(["dpd_bucket_"+str(i)+" >= 2" for i in range(1,7)])+" ) then 0 \
    else null end as satisfactory_flag_6m"
]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" =0" for i in range(1,25)])+" then 1 else 0 end as dpd0"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=0" for i in range(1,25)])+" then 1 else 0 end as dpdvalid"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=0" for i in range(1,13)])+" then 1 else 0 end as dpdvalid_12m"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=1" for i in range(1,25)])+" then 1 else 0 end as dpdx"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=2" for i in range(1,25)])+" then 1 else 0 end as dpd30"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=3" for i in range(1,25)])+" then 1 else 0 end as dpd60"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=4" for i in range(1,25)])+" then 1 else 0 end as dpd90"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=5" for i in range(1,25)])+" then 1 else 0 end as dpd150"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=6" for i in range(1,25)])+" then 1 else 0 end as dpd180"]\
+["case when "+" or ".join(["dpd_bucket_"+str(i)+" >=7" for i in range(1,25)])+" then 1 else 0 end as dpd360"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" ==0)" for i in range(1,25)])+" as dpd0_counter"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=0)" for i in range(1,25)])+" as dpdvalid_counter"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=1)" for i in range(1,25)])+" as dpdx_counter"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=2)" for i in range(1,25)])+" as dpd30_counter"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=3)" for i in range(1,25)])+" as dpd60_counter"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=4)" for i in range(1,25)])+" as dpd90_counter"]\
+["case "+" ".join(["when dpd_bucket_"+str(i)+" >0 then "+str(i) for i in range(1,37)])+" else null end as min_dpd_month"]\
+["case "+" ".join(["when dpd_bucket_"+str(i)+" >0 then "+str(i) for i in range(36,0,-1)])+" else null end as max_dpd_month"]\
+["greatest("+", ".join(["dpd_bucket_"+str(i) for i in range(1,25)])+") as largest_delinquency"]\
+["case when UNSECURED=1 then SANCTIONAMOUNT end as unsecured_sanamt"]\
+["case when SECURED=1 then SANCTIONAMOUNT end as secured_sanamt"]\
+["case when Open=1 and PRODUCT_DESC='{prod}' then mn_overdue else 0 end as {prod}_AmtpastDue_open".format(prod=prod) for prod in prods]\
+["case when Installment=1 then mn_overdue else 0 end as amt_past_due_install"]\
+["case when Open=1 then mn_overdue else 0 end as AmtpastDue_open"]\
+["case when open=1 and PRODUCT_DESC='"+prod+"' then LASTPAYMENT else null end as "+prod+"_open_paymnt" for prod in prods]\
+["case when open=1 then LASTPAYMENT else 0 end as open_payment"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=3)" for i in range(1,7)])+" as dpd60_counter_last_6m"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=4)" for i in range(1,7)])+" as dpd90_counter_last_6m"]\
+[" + ".join(["int(dpd_bucket_"+str(i)+" >=4)" for i in range(1,13)])+" as dpd90_counter_last_12m"]\
+["greatest("+", ".join(["dpd_bucket_"+str(i) for i in range(1,4)])+") as max_dpd_3m"]\
+["greatest("+", ".join(["dpd_bucket_"+str(i) for i in range(1,7)])+") as max_dpd_6m"]\
+["greatest("+", ".join(["dpd_bucket_"+str(i) for i in range(1,13)])+") as max_dpd_12m"]\
+["case when PRODUCT_DESC='CC' then greatest("+", ".join(["dpd_bucket_"+str(i) for i in range(1,37)])+") else null end as max_dpd_cc_ever"]\
+["case when PRODUCT_DESC='PL' then SANCTIONAMOUNT else null end as PL_sanc_amt"]\
+["case when open=1 and PRODUCT_DESC='CC' then coalesce(CREDITLIMIT,greatest(SANCTIONAMOUNT,HIGHCREDIT)) else null end as CC_sanc_amt"]\
+["case when PRODUCT_DESC='PL' and mn_diff_retro_open>=1 and mn_diff_retro_open<=12 and SANCTIONAMOUNT>=30000 and SANCTIONAMOUNT<=1000000 then 1 else 0 end as pl_open_12"]\
+["case when ACCOUNT_TYPE in ('Housing Loan') and SANCTIONAMOUNT > 15000 and tenure <= 36 then 1 else 0 end as HL_gt_15k_36mob_flag"]\
+["case when BALANCE is null or CREDITLIMIT is null or CREDITLIMIT = 0 then 0 \
when ACCOUNT_TYPE in ('Corporate Credit Card','Credit Card','Secured Credit Card') and BALANCE/CREDITLIMIT < 0.6 and tenure <= 12 then 1 else 0 end as CC_util_lt_60_12mob_flag"]\
+["case when PRODUCT_DESC = 'PL' and tenure <= 12 then 1 else 0 end as pl_open_12m_flag"]

In [14]:
tl_exp8 = ["*"]\
+["case when satisfactory_flag_6m = 1 then BALANCE else null end as satisfactory_bal_6m"]\
+["case when satisfactory_flag_12m = 1 then BALANCE else null end as satisfactory_bal_12m"]\
+["case when satisfactory_flag_24m = 1 then BALANCE else null end as satisfactory_bal_24m"]\
+["case when wof_flag = 1 then BALANCE else null end as derog_bal"]\
+[
    "case when wof_flag = 1 and PRODUCT_DESC='{prod}' then BALANCE else null end as {prod}_derog_bal".format(prod=prod)
    for prod in prods
]\
+[
    "case when Open = 1 and dpd_bucket_1 > 1 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_open_curr_30del".format(prod=prod)
    for prod in ['BL' , 'PL' , 'CC' , 'TW']
]\
+[
    "case when Open = 1 and dpd_bucket_1 > 2 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_open_curr_60del".format(prod=prod) 
    for prod in ['BL' , 'PL' , 'CC' , 'TW']
]\
+[
    "case when Open = 1 and dpd_bucket_1 > 3 and PRODUCT_DESC='{prod}' then 1 else null end as {prod}_open_curr_90del".format(prod=prod) 
    for prod in ['BL' , 'PL' , 'CC' , 'TW']
]\
+[
    "case when tenure <= 24 and PRODUCT_DESC='{prod}' then 1 else 0 end as {prod}_trades_24_months".format(prod=prod) 
    for prod in prods
]\
+[
    "case when tenure <= 24 and Open = 1 and PRODUCT_DESC='{prod}' then BALANCE else null end as {prod}_open_balance_24m".format(prod=prod) 
    for prod in prods
]\
+["case when dpdx = 1 then mn_overdue else null end as past_due_dpdx_trade"]\
+["case when Open=1 and dpdx = 1 then mn_overdue else null end as open_past_due_dpdx_trade"]\
+["case when ACCOUNT_TYPE = 'Credit Card' then mn_overdue else null end as past_due_cc"]\
+["case when ACCOUNT_TYPE = 'Credit Card' then BALANCE else null end as balance_cc"]\
+["case when satisfactory_flag_6m = 0 then BALANCE else null end as unsat_bal_6m"]\
+["case when satisfactory_flag_12m = 0 then BALANCE else null end as unsat_bal_12m"]\
+["case when satisfactory_flag_24m = 0 then BALANCE else null end as unsat_bal_24m"]\
+["case when Open = 1 and satisfactory_flag_24m = 1 then BALANCE else null end as open_sat_bal_24m"]\
+["case when satisfactory_flag_24m = 1 then mn_overdue else null end as sat_due_amt_24m"]\
+["case when satisfactory_flag_24m = 0 then mn_overdue else null end as unsat_due_amt_24m"]\
+["case when satisfactory_flag_24m = 0 then 1 else 0 end as unsat_trade"]\
+["case when Open = 1 and dpd_bucket_1 > 1 then 1 else null end as open_curr_30del"]\
+["case when Open = 1 and dpd_bucket_1 > 2 then 1 else null end as open_curr_60del"]\
+["case when Open = 1 and dpd_bucket_1 > 3 then 1 else null end as open_curr_90del"]\
+["case when tenure <= 24 then 1 else 0 end as trades_24_months"]\
+["case when tenure <= 24 and Open = 1 then BALANCE else null end as open_balance_24m"]\
+["case when ACCOUNT_TYPE = 'Credit Card' then tenure else null end as tenure_cc"]\
+["case when Open = 1 then tenure else null end as open_tenure"]\
+["case when Open = 1 and satisfactory_flag_24m = 1 then tenure else null end as open_sat_tenure"]\
+["case when Revolving=1 then (Revolving_credit_limit-BALANCE) else null end as revol_Balance_Avail"]\
+["case "+" ".join(["when dpd_bucket_"+str(i)+" = largest_delinquency then "+str(i) for i in range(1,37)])+" end as highest_dpd_month"]\
+["month_scrub_payhist_start + min_dpd_month as month_since_first_del"]\
+["month_scrub_payhist_start + max_dpd_month as month_since_last_del"]\
+["case when tenure >= 12 then SANCTIONAMOUNT else 0 end as TRD_12_MN_SAN"]\
+["case when SANCTIONAMOUNT <=1000000 then tenure else null end as tenure_10L"]\
+["case when PRODUCT_DESC in ('PL','BL') and open =1 then BALANCE else 0 end as plbl_bal"]\
+["case when PRODUCT_DESC in ('PL','BL') then tenure else null end as plbl_tenure"]\
+["case when open=1 and dpdx=1 then BALANCE else null end as open_dpdx_bal"]\
+["case when open=1 and dpdx=1 then emi else null end as open_dpdx_emi"]\
+["case when SECURED = 1 and dpd60_counter_last_6m >= 1 then 1 else 0 end as dpd60_last_6m_secured"]\
+["case when SECURED = 0 and dpd60_counter_last_6m >= 1 then 1 else 0 end as dpd60_last_6m_unsecured"]\
+["case when SECURED = 1 then dpd90_counter_last_6m else 0 end as dpd90_counter_last_6m_secured"]\
+["case when SECURED = 0 then dpd90_counter_last_6m else 0 end as dpd90_counter_last_6m_unsecured"]\
+["case when dpd90_counter_last_12m > 0 then 1 else 0 end as ever_dpd90_last_12m"]

In [15]:
tl_exp9=["*"]\
+["month_scrub_payhist_start + highest_dpd_month as month_since_highest_del"]\
+["case when is_open_m0 = 1 and max_dpd_reported_6m > 0 then max_dpd_reported_6m end as live_acct_max_dpd_reported_6m"]\
+[
    f"case when is_open_m0 = 1 and product_desc = '{prod}' and max_dpd_reported_6m > 0 then max_dpd_reported_6m end as live_{prod}_acct_max_dpd_reported_6m"
    for prod in ['PL' , 'CD']
]\
+[
    f"case when is_open_m0 = 1 and secured = {val[0]} and max_dpd_reported_6m > 0 then max_dpd_reported_6m end as live_{val[1]}_acct_max_dpd_reported_6m"
    for val in [(1 , 'sec') , (0 , 'unsec')]
]

In [16]:
tl_agg_exp = []\
+[F.sum(F.col('Trade')).alias('cv_var_trd')]\
+[F.sum(F.col('Installment')).alias('cv_var_inst_trd')]\
+[F.sum(F.col('cc_overdue')).alias('sum_cc_overdue')]\
+[F.sum(F.col('non_cc_overdue')).alias('sum_non_cc_overdue')]\
+[F.sum(F.col('open_overdue')).alias('sum_open_overdue')]\
+[F.sum(F.col('mn_overdue')).alias('sum_mn_overdue')]\
+[F.sum(F.col('cc_relax_dpd_overdue')).alias('sum_cc_relax_dpd_overdue')]\
+[F.sum(F.col('Installment')).alias('Trd_num_installment')]\
+[F.sum(F.col('Open_installment')).alias('Trd_num_open_installment')]\
+[F.sum(F.col('Install_balance')).alias('Trd_sum_install_balance')]\
+[F.sum(F.col('Open_install_balance')).alias('Trd_sum_open_install_balance')]\
+[F.sum(F.col('Monthly_payment_amt_install')).alias('Trd_sum_Monthly_payment_amt_install')]\
+[F.sum(F.col('Monthly_payment_amt_open_install')).alias('Trd_sum_Monthly_payment_amt_open_install')]\
+[F.sum(F.col('Loan_amt_open_install')).alias('Trd_sum_Loan_amt_open_install')]\
+[F.sum(F.col('amt_past_due_install')).alias('Trd_sum_amt_past_due_install')]\
+[F.sum(F.col('Revolving')).alias('Trd_num_revolving')]\
+[F.sum(F.col('Open_revolving')).alias('Trd_num_Open_revolving')]\
+[F.sum(F.col('Revolving_balance_amt')).alias('Trd_sum_Revolving_balance_amt')]\
+[F.sum(F.col('Open_Revolving_balance_amt')).alias('Trd_sum_Open_Revolving_balance_amt')]\
+[F.sum(F.col('Monthly_payment_revolving')).alias('Trd_sum_Monthly_payment_revolving')]\
+[F.sum(F.col('Revolving_credit_limit')).alias('Trd_sum_Revolving_credit_limit')]\
+[F.sum(F.col('Revolving_open_credit_limit')).alias('Trd_sum_Revolving_open_credit_limit')]\
+[F.sum(F.col('SECURED')).alias('Trd_num_secured_trade')]\
+[F.sum(F.col('UNSECURED')).alias('Trd_num_unsecured_trade')]\
+[F.sum(F.col('secured_open_emi')).alias('Trd_sum_secured_emi')]\
+[F.sum(F.col('unsecured_open_emi')).alias('Trd_sum_unsecured_open_emi')]\
+[F.sum(F.col("EMI")).alias("Trd_sum_EMI")]\
+[F.sum(F.col('open_balance')).alias('Trd_sum_open_balance')]\
+[F.sum(F.col('mn_overdue')).alias('Trd_sum_amt_past_due_all')]\
+[F.sum(F.col("BALANCE")).alias("Trd_sum_BALANCE")]\
+[F.max(F.col('BALANCE')).alias('Trd_max_balance_amt')]\
+[F.max(F.col('mn_overdue')).alias('Trd_max_past_due_amt')]\
+[F.max(F.col('SANCTIONAMOUNT')).alias('Trd_max_loan_amt')]\
+[F.max(F.col('CREDITLIMIT')).alias('Trd_max_credit_limit')]\
+[F.sum(F.col('Open')).alias('Trd_num_open_trade')]\
+[F.sum(F.col('Monthly_payment_open')).alias('Trd_Sum_Monthly_payment_open')]\
+[F.sum(F.col('AmtpastDue_open')).alias('Trd_sum_AmtpastDue_open')]\
+[F.sum(F.col(prod+'_AmtpastDue_open')).alias(prod+'_sum_AmtpastDue_open') for prod in prods]\
+[F.sum(F.col('tenure')).alias('Trd_sum_tenure')]\
+[F.max(F.col('tenure')).alias('Trd_max_tenure')]\
+[F.avg(F.col('tenure')).alias('Trd_avg_tenure')]\
+[F.min(F.col('tenure')).alias('Trd_min_tenure')]\
+[F.sum(F.col(prod+'_tenure')).alias(prod+'_sum_tenure') for prod in prods]\
+[F.sum(F.col("LOAN_AMOUNT")).alias("Trd_sum_LOAN_AMOUNT")]\
+[F.sum(F.col("CREDITLIMIT")).alias("Trd_sum_CREDITLIMIT")]\
+[F.sum(F.col("HIGHCREDIT")).alias("Trd_sum_HIGHCREDIT")]\
+[F.max(F.col("HIGHCREDIT")).alias("Trd_max_HIGHCREDIT")]\
+[F.avg(F.col("sec_tenure")).alias("Trd_avg_sec_tenure")]\
+[F.min(F.col("msp")).alias("Trd_month_since_last_payment")]\
+[F.sum(F.col("curr_satisfactory")).alias("Trd_Sum_Curr_satisfactory")]\
+[F.sum(F.col("satisfactory_bal_6m")).alias("Trd_Sum_Sat_Balance_6M")]\
+[F.sum(F.col("satisfactory_bal_12m")).alias("Trd_Sum_Sat_Balance_12M")]\
+[F.sum(F.col("satisfactory_bal_24m")).alias("Trd_Sum_Sat_Balance_24M")]\
+[F.avg(F.col("satisfactory_bal_6m")).alias("Trd_avg_Sat_Balance_6M")]\
+[F.avg(F.col("satisfactory_bal_12m")).alias("Trd_avg_Sat_Balance_12M")]\
+[F.avg(F.col("satisfactory_bal_24m")).alias("Trd_avg_Sat_Balance_24M")]\
+[F.sum(F.col("satisfactory_flag_24m")).alias("Trd_Sum_Satisfactory_trade")]\
+[F.sum(F.col("past_due_cc")).alias("Trd_Sum_past_due_CC")]\
+[F.sum(F.col("balance_cc")).alias("Trd_Sum_balance_CC")]\
+[F.sum(F.col("unsat_trade")).alias("Trd_Sum_Unsatisfactory_trades")]\
+[F.sum(F.col("unsat_bal_6m")).alias("Trd_Sum_Unsat_bal_6M")]\
+[F.sum(F.col("unsat_bal_12m")).alias("Trd_Sum_Unsat_bal_12M")]\
+[F.sum(F.col("unsat_bal_24m")).alias("Trd_Sum_Unsat_bal_24M")]\
+[F.avg(F.col("unsat_bal_6m")).alias("Trd_avg_Unsat_bal_6M")]\
+[F.avg(F.col("unsat_bal_12m")).alias("Trd_avg_Unsat_bal_12M")]\
+[F.avg(F.col("unsat_bal_24m")).alias("Trd_avg_Unsat_bal_24M")]\
+[F.sum(F.col("open_sat_bal_24m")).alias("Trd_Sum_open_sat_balance")]\
+[F.avg(F.col("open_sat_bal_24m")).alias("Trd_Avg_open_sat_balance")]\
+[F.sum(F.col("open_curr_30del")).alias("Trd_sum_open_trades_curr_30del")]\
+[F.sum(F.col("open_curr_60del")).alias("Trd_sum_open_trades_curr_60del")]\
+[F.sum(F.col("open_curr_90del")).alias("Trd_sum_open_trades_curr_90del")]\
+[F.sum(F.col(prod+"_open_curr_30del")).alias(prod+"_sum_open_trades_curr_30del") for prod in ['BL' , 'PL' , 'CC' , 'TW']]\
+[F.sum(F.col(prod+"_open_curr_60del")).alias(prod+"_sum_open_trades_curr_60del") for prod in ['BL' , 'PL' , 'CC' , 'TW']]\
+[F.sum(F.col(prod+"_open_curr_90del")).alias(prod+"_sum_open_trades_curr_90del") for prod in ['BL' , 'PL' , 'CC' , 'TW']]\
+[F.max(F.col("dpd_bucket_1")).alias("Trd_highest_del_in_curr")]\
+[F.sum(F.col("sat_due_amt_24m")).alias("Trd_sum_past_due_sat_trade")]\
+[F.avg(F.col("sat_due_amt_24m")).alias("Trd_avg_past_due_sat_trade")]\
+[F.sum(F.col("unsat_due_amt_24m")).alias("Trd_sum_past_due_unsat_trade")]\
+[F.avg(F.col("unsat_due_amt_24m")).alias("Trd_avg_past_due_unsat_trade")]\
+[F.sum(F.col("trades_24_months")).alias("Trd_sum_trades_24M")]\
+[F.sum(F.col("open_balance_24m")).alias("Trd_sum_open_balance_24M")]\
+[F.sum(F.col(prod+"_trades_24_months")).alias(prod+"_sum_trades_24M") for prod in prods]\
+[F.sum(F.col(prod+"_open_balance_24m")).alias(prod+"_sum_open_balance_24M") for prod in prods]\
+[F.sum(F.col("secured_balance")).alias("Trd_sum_secured_balance")]\
+[F.sum(F.col("unsecured_balance")).alias("Trd_sum_unsecured_balance")]\
+[F.sum(F.col("open_balance_3m")).alias("Trd_sum_open_balance_3m")]\
+[F.sum(F.col(prod+"_open_balance_3m")).alias(prod+"_sum_open_balance_3m") for prod in prods]\
+[F.sum(F.col("open_balance_6m")).alias("Trd_sum_open_balance_6m")]\
+[F.sum(F.col(prod+"_open_balance_6m")).alias(prod+"_sum_open_balance_6m") for prod in prods]\
+[F.count(F.col("num_open_last1m")).alias("Trd_num_open_last1m")]\
+[F.count(F.col("num_open_last3m")).alias("Trd_num_open_last3m")]\
+[F.count(F.col("num_open_last6m")).alias("Trd_num_open_last6m")]\
+[F.count(F.col("num_open_last12m")).alias("Trd_num_open_last12m")]\
+[F.count(F.col(prod+"_num_open_last1m")).alias(prod+"_num_open_last1m") for prod in prods]\
+[F.count(F.col(prod+"_num_open_last3m")).alias(prod+"_num_open_last3m") for prod in prods]\
+[F.count(F.col(prod+"_num_open_last6m")).alias(prod+"_num_open_last6m") for prod in prods]\
+[F.count(F.col(prod+"_num_open_last12m")).alias(prod+"_num_open_last12m") for prod in prods]\
+[F.max(F.col("tenure_open_install")).alias("Trd_max_tenure_open_install")]\
+[F.avg(F.col("tenure_open_install")).alias("Trd_avg_tenure_open_install")]\
+[F.avg(F.col("tenure_open_revol")).alias("Trd_avg_tenure_open_revol")]\
+[F.max(F.col("tenure_open_revol")).alias("Trd_max_tenure_open_revol")]\
+[F.avg(F.col("tenure_all_install")).alias("Trd_avg_tenure_all_install")]\
+[F.max(F.col("tenure_all_install")).alias("Trd_max_tenure_all_install")]\
+[F.avg(F.col("tenure_all_revol")).alias("Trd_avg_tenure_all_revol")]\
+[F.max(F.col("tenure_all_revol")).alias("Trd_max_tenure_all_revol")]\
+[F.max(F.col("tenure_cc")).alias("Trd_max_tenure_cc")]\
+[F.max(F.col("open_tenure")).alias("Trd_max_tenure_open_trades")]\
+[F.max(F.col("open_sat_tenure")).alias("Trd_max_tenure_open_sat")]\
+[F.min(F.col("open_tenure")).alias("Trd_month_since_most_recent_trade")]\
+[F.sum(F.col("revol_Balance_Avail")).alias("Trd_Sum_avail_bal_revol")]\
+[F.min(F.col("month_since_first_del")).alias("Trd_month_since_first_del")]\
+[F.max(F.col("month_since_last_del")).alias("Trd_month_since_last_del")]\
+[F.max(F.col("month_since_highest_del")).alias("Trd_months_since_max_del_ever")]\
+[F.sum("dpd0").alias("dpd0")]\
+[F.sum("dpdx").alias("dpdx")]\
+[F.sum("dpd30").alias("dpd30")]\
+[F.sum("dpd60").alias("dpd60")]\
+[F.sum("dpd90").alias("dpd90")]\
+[F.sum("dpdvalid").alias("dpdvalid")]\
+[F.sum("dpd0_counter").alias("dpd0_counter")]\
+[F.sum("dpdx_counter").alias("dpdx_counter")]\
+[F.sum("dpd30_counter").alias("dpd30_counter")]\
+[F.sum("dpd60_counter").alias("dpd60_counter")]\
+[F.sum("dpd90_counter").alias("dpd90_counter")]\
+[F.sum("dpdvalid_counter").alias("dpdvalid_counter")]\
+[F.sum("derog_bal").alias("derog_bal")]\
+[F.sum(prod+"_derog_bal").alias(prod+"_derog_bal") for prod in prods]\
+[F.sum(prod+"_ind").alias(prod+"_ind") for prod in prods]\
+[F.sum(prod+"_bal").alias (prod+"_bal") for prod in prods]\
+[F.avg(prod+"_bal").alias(prod+"_bal_avg") for prod in prods]\
+[F.sum(prod+"_emi").alias(prod+"_emi") for prod in prods]\
+[F.sum(prod+"_open_paymnt").alias(prod+"_open_paymnt_sum") for prod in prods]\
+[F.sum(prod+"_sanc_amt_6m").alias(prod+"_sanc_amt_6m_sum") for prod in prods]\
+[F.first("first_product").alias("first_product")]\
+[F.first("latest_product").alias("latest_product")]\
+[F.sum("EMI").alias("total_emi")]\
+[F.sum("TRD_12_MN_SAN").alias("TRD_12_MN_SAN")]\
+[F.sum("tenure_10L").alias("tenure_10L")]\
+[F.sum("unsecured_sanamt").alias("unsecured_sanamt")]\
+[F.sum("secured_sanamt").alias("secured_sanamt")]\
+[F.sum("plbl_bal").alias("plbl_bal")]\
+[F.avg("plbl_tenure").alias("plbl_tenure")]\
+[F.sum("open_payment").alias("open_payment")]\
+[F.sum("open_dpdx_bal").alias("open_dpdx_bal")]\
+[F.sum("open_dpdx_emi").alias("open_dpdx_emi")]\
+[F.max("dpd60_last_6m_secured").alias("dpd60_last_6m_secured")]\
+[F.max("dpd60_last_6m_unsecured").alias("dpd60_last_6m_unsecured")]\
+[F.max("dpd90_counter_last_6m_secured").alias("count_dpd90_last_6m_secured")]\
+[F.max("dpd90_counter_last_6m_unsecured").alias("count_dpd90_last_6m_unsecured")]\
+[F.max("wof_flag").alias("max_wof_flag")]\
+[F.sum("wof_flag").alias("cnt_accts_wof_flag")]\
+[
    F.expr(
        f"""
        sum(wof_flag * int(product_desc == '{prod}')) as cnt_{prod}_accts_wof_flag 
        """
    )
    for prod in ['HL' , 'AL' , 'CC' , 'PL' , 'BL' , 'CD']
]\
+[
    F.expr(
        f"""
        sum(wof_flag * int({flag} == 1)) as cnt_{flag}_accts_wof_flag 
        """
    )
    for flag in ['secured' , 'unsecured']
]\
+[F.max("ever_dpd90_last_12m").alias("ever_dpd90_last_12m")]\
+[F.max('max_dpd_3m').alias('max_dpd_3m')]\
+[F.max('max_dpd_cc_ever').alias('max_dpd_cc_ever')]\
+[F.countDistinct('ACCOUNT_TYPE').alias('no_distinct_prdcts')]\
+[F.sum('PL_sanc_amt').alias('PL_sanc_amt')]\
+[F.sum('CC_sanc_amt').alias('CC_sanc_amt')]\
+[F.max('max_dpd_6m').alias('max_dpd_6m')]\
+[F.max('max_dpd_12m').alias('max_dpd_12m')]\
+[F.max('pl_open_12').alias('pl_open_12_30k_10l')]\
+[F.min('months_closed').alias('inactive_flag_last12m')]\
+[
    F.count(f'live_{prod}acct_max_dpd_reported_6m').alias(f'num_live_{prod}accts_max_dpd_reported_0plus_6m')
    for prod in ['' , 'PL_' , 'CD_' , 'sec_' , 'unsec_']
]\
+[
    F.max(f'live_{prod}acct_max_dpd_reported_6m').alias(f'max_dpd_across_live_{prod}accts_max_dpd_reported_0plus_6m')
    for prod in ['' , 'PL_' , 'CD_' , 'sec_' , 'unsec_']
]\
+[
    F.expr(
        f"""
        sum({var_type}emi_m0 * max_dpd_reported_6m_flag) as total_{var_type}emi_m0_max_dpd_reported_0plus_6m
        """
    )
    for var_type in [
        '' , 'lt_' , 'st_', 'secure_' , 'unsecure_' , 'revolve_' , 'install_' , 'CD_' ,
        'AL_' , 'BL_' , 'PL_' , 'HL_' , 'CC_' , 'PL_short_' , 'PL_long_' 
    ]
]\
+[F.sum('dpd_curr').alias('num_acct_curr_dpd')]\
+[F.max('dpd_curr').alias('curr_act_dpd_flag')]\
+[F.sum("negative_balance_flag").alias("bal_neg_cnt")]\
+[F.sum('HL_gt_15k_36mob_flag').alias('HL_gt_15k_36mob')]\
+[F.sum('CC_util_lt_60_12mob_flag').alias('CC_util_lt_60_12mob')]\
+[F.sum('pl_open_12m_flag').alias('pl_open_12m')]\
+[F.min('mn_diff_retro_start').alias('min_reporting')]\
+[F.sum('is_open_m0_m2').alias('num_loans_live_m0_m2')]\
+[F.sum('is_open_m6_m8').alias('num_loans_live_m6_m8')]

In [17]:
tl_exp10 = ["*"]\
+[
    """
    case when open_dpdx_emi is null then -1
    when open_dpdx_bal is null then -2 
    when open_dpdx_bal = 0 then -3
    else open_dpdx_emi / open_dpdx_bal end as Trd_open_dpdx_ratio_emi_bal
    """
]\
+[
    """
    case when Trd_sum_EMI is null then -1
    when Trd_sum_open_balance is null then -2 
    when Trd_sum_open_balance = 0 then -3
    else Trd_sum_EMI / Trd_sum_open_balance end as Trd_ratio_emi_bal
    """
]\
+[
    """
    case when Trd_sum_Revolving_balance_amt is null then -1
    when Trd_sum_Revolving_credit_limit is null then -2
    when Trd_sum_Revolving_credit_limit = 0 then -3
    else Trd_sum_Revolving_balance_amt / Trd_sum_Revolving_credit_limit end as Trd_ratio_bal_creditlimit_revol
    """
]\
+[
    """case when Trd_sum_Open_Revolving_balance_amt is null then -1
    when Trd_sum_Revolving_open_credit_limit is null then -2
    when Trd_sum_Revolving_open_credit_limit = 0 then -3
    else Trd_sum_Open_Revolving_balance_amt / Trd_sum_Revolving_open_credit_limit end as Trd_ratio_open_bal_creditlimit_revol
    """
]\
+[
    """
    case when Trd_sum_Open_Revolving_balance_amt is null then -1
    else 0.05 * Trd_sum_Open_Revolving_balance_amt end as Trd_5per_revol_balance
    """
]\
+[
    """
    case when Trd_sum_open_install_balance is null then -1
    when Trd_sum_Loan_amt_open_install is null then -2
    when Trd_sum_Loan_amt_open_install = 0 then -3
    else Trd_sum_open_install_balance / Trd_sum_Loan_amt_open_install end as Trd_ratio_open_bal_loanamt_instal
    """
]\
+[
    """
    case when Trd_sum_install_balance is null then -1
    when Trd_sum_LOAN_AMOUNT is null then -2
    when Trd_sum_LOAN_AMOUNT = 0 then -3
    else Trd_sum_install_balance / Trd_sum_LOAN_AMOUNT end as Trd_ratio_bal_loanamt_instal
    """
]\
+[
    """
    case when Trd_sum_amt_past_due_all is null then -1
    when Trd_sum_BALANCE is null then -2
    when Trd_sum_BALANCE = 0 then -3
    else Trd_sum_amt_past_due_all / Trd_sum_BALANCE end as Trd_ratio_amtpastdue_bal
    """
]\
+ [
    """
    case when Trd_Sum_open_sat_balance is null then -1
    when Trd_sum_open_balance_24M is null then -2
    when Trd_sum_open_balance_24M = 0 then -3
    else Trd_Sum_open_sat_balance/Trd_sum_open_balance_24M end as Trd_ratio_open_sat_total_bal_24M
    """
]\
+ [
    """
    case when Trd_sum_secured_balance is null then -1
    when Trd_sum_unsecured_balance is null then -2
    when Trd_sum_unsecured_balance = 0 then -3
    else Trd_sum_secured_balance / Trd_sum_unsecured_balance end as Trd_ratio_secured_unsecured_balances
    """
]\
+ [
    """
    case when Trd_sum_open_balance_3m is null then -1 
    when Trd_sum_open_balance is null then -2
    when Trd_sum_open_balance = 0 then -3
    else Trd_sum_open_balance_3m / Trd_sum_open_balance end as Trd_per_outstanding_3m
    """
]\
+ [
    """
    case when Trd_sum_open_balance_6m is null then -1
    when Trd_sum_open_balance is null then -2
    when Trd_sum_open_balance = 0 then -3
    else Trd_sum_open_balance_6m / Trd_sum_open_balance end as Trd_per_outstanding_6m
    """
]\
+ [
    """
    case when Trd_Sum_avail_bal_revol is null then -1
    when Trd_sum_CREDITLIMIT is null then -2
    when Trd_sum_CREDITLIMIT = 0 then -3
    else Trd_Sum_avail_bal_revol / Trd_sum_CREDITLIMIT end as Trd_perc_avail_revol_cc
    """
]\
+["dpd"+x+"/dpdvalid as frac_dpd_trades_"+x for x in ["0","x","30","60","90"]]\
+["dpd"+x+"_counter/dpdvalid_counter as frac_dpd_trademonths_"+x for x in ["0","x","30","60","90"]]\
+[
    """
    case when Trd_sum_amt_past_due_all >= 1000 then 1
    else 0 end as sum_amt_past_due_gt_1k 
    """
]\
+[
    """
    case when PL_sanc_amt = 0 then -2
    when PL_sanc_amt is null then -1
    else (PL_sanc_amt - PL_bal) / PL_sanc_amt end as avg_PL_bal_runoff
    """
]\
+[
    """
    case when CC_sanc_amt = 0 then -2
    when CC_sanc_amt is null then -1
    else CC_bal / CC_sanc_amt end as CC_Utilization
    """
]\
+["num_loans_live_m0_m2 - num_loans_live_m6_m8 as diff_num_loans_live_m0_m2_and_m6_m8"]

In [18]:
tl_exp11=["*"]\
+['case when CC_Utilization >= 0.85 then 1 else 0 end as cc_util_gt_85']

In [19]:
trd = spark.read.parquet(prep_data_path + "prep_tl_data.parquet/")

In [20]:
trd.show(1, False, True)

25/05/09 07:52:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 EnquiryControlNumber            | 99401699                                                                                                                                                                                  
 MemberReference                 | 79238541                                                                                                                                                                                  
 acct_uniq_id                    | NOT DISCLOSED                                                                                                                                                                             
 Loan_Status                     | Closed                                                                       

In [21]:
trd4 = trd\
.selectExpr(tl_exp5)\
.withColumn('first_product', F.first(F.col('PRODUCT_DESC')).over(w1))\
.withColumn('latest_product', F.first(F.col('PRODUCT_DESC')).over(w2))\
.withColumn('row_num' , F.row_number().over(w3))\
.selectExpr(tl_exp6)\
.withColumn(
    'HIGHCREDIT', 
    F.when(F.col('PRODUCT_DESC') == 'CC', F.greatest(F.col('HIGHCREDIT'),F.col('CREDITLIMIT')))\
    .otherwise(F.col('HIGHCREDIT'))
)\
.withColumn('arr' , F.array(*[f'dpd_bucket_{i}' for i in range(1 , 37)]).cast(T.ArrayType(T.IntegerType())))\
.withColumn('cnt_0' , F.size(F.col('arr')) - F.size(F.array_remove('arr' , 0)))\
.withColumn('cnt_minus2' , F.size(F.col('arr')) - F.size(F.array_remove('arr' , -2)))\
.withColumn('dpd_curr' , F.when(F.col('cnt_0') + F.col('cnt_minus2') == 36 , F.lit(1)).otherwise(F.lit(0)))\
.selectExpr(leverage_exp1)\
.selectExpr(leverage_exp2)\
.selectExpr(leverage_exp3)\
.selectExpr(leverage_exp4)\
.selectExpr(leverage_exp5)\
.selectExpr(leverage_exp6)\
.selectExpr(tenure_exp1)\
.withColumn(
    "loan_tenure" , 
    F.when(
        F.isnull(F.col('ACCOUNT_CLOSING_DATE')) == False ,
        F.months_between(F.date_trunc('month' , F.col('ACCOUNT_CLOSING_DATE')) , F.date_trunc('month' , F.col('DATEOPENED'))) + F.lit(1)
    ).when(
        (F.isnull(F.col('repaymenttenure')) == False) & (F.col('repaymenttenure') > 0),
        F.col('repaymenttenure') ,
    ).when(
        (F.col('BALANCE') > 0) & (F.col('EMI_NEW') > 0) ,
        F.col('mn_diff_retro_open') + (F.col('BALANCE') / F.col('EMI_NEW')) ,
    ).otherwise(
        (F.col('SANCTIONAMOUNT_NEW') / F.col('EMI_NEW'))
    )
)\
.withColumn(
    "mn_retro_acct_close" , 
    F.when(
        F.isnull(F.col('ACCOUNT_CLOSING_DATE')) == False ,
        F.months_between(F.col('run_dt') , F.date_trunc('month' , F.col('ACCOUNT_CLOSING_DATE')))
    ).otherwise(
        F.lit(None)
    )
)\
.selectExpr(tenure_exp2)\
.selectExpr(tenure_exp3)\
.selectExpr(loan_status_exp1)\
.selectExpr(loan_status_exp2)\
.selectExpr(loan_status_exp3)\
.withColumn('arr' , F.array(*[f'phist_bucket_{i}' for i in range(1 , 7)]).cast(T.ArrayType(T.IntegerType())))\
.withColumn('max_dpd_reported_6m' , F.array_max(F.col('arr')))\
.withColumn('max_dpd_reported_6m_flag' , F.when(F.col('max_dpd_reported_6m') >= 1 , F.lit(1)).otherwise(F.lit(0)))\
.withColumn('is_open_m0_m2' , F.greatest(F.col('is_open_m0') , F.col('is_open_m1') , F.col('is_open_m2')))\
.withColumn('is_open_m6_m8' , F.greatest(F.col('is_open_m6') , F.col('is_open_m7') , F.col('is_open_m8')))

In [22]:
trd4.filter('PRODUCT_DESC = "Other"').select('account_type').distinct().show(truncate = False)

+------------+
|account_type|
+------------+
|Other       |
+------------+



In [23]:
trd4.filter('is_open_m0 != open').count()

0

In [24]:
trd5 = trd4\
.selectExpr(tl_exp_new0)\
.selectExpr(tl_exp_new)\
.selectExpr(tl_exp7)\
.selectExpr(tl_exp8)\
.selectExpr(tl_exp9)

In [25]:
final_agg = tl_agg_exp + leverage_agg_exp1 + tenure_agg_exp1 + loan_status_agg_exp1 

In [26]:
trd6 = trd5.groupby("MemberReference").agg(*final_agg)\
.withColumn('max_num_concurrent_live_loans_m0_m12' , F.greatest(*[F.col(f"num_concurrent_live_loans_m{i}") for i in range(0, 13)]))\
.withColumn('max_emi_m0_m12' , F.greatest(*[F.col(f"sum_emi_m{i}_vintage_all_mn") for i in range(0, 13)]))

In [27]:
trd7 = trd6\
.selectExpr(tl_exp10)\
.selectExpr(tl_exp11)\
.selectExpr(leverage_agg_exp2)\
.selectExpr(leverage_agg_exp3)\
.selectExpr(loan_status_agg_exp2)\
.selectExpr(loan_status_agg_exp3)\
.drop(
    *(
        [F.col(f"sum_{loan_type}emi_m{i}_vintage_all_mn") for i in range(1, 13) for loan_type in ('' , 'lt_' , 'st_' , 'secure_')] + 
        [F.col(f"num_concurrent_live_loans_m{i}") for i in range(1, 13)] + 
        [F.col(f"avg_emi_per_live_loans_m{i}_vintage_all_mn") for i in range(1, 13)]
    )
)

In [28]:
trd8 = trd7.toDF(
    *[
        col.lower() 
        if ((col.lower() in (['id'])) or (col.startswith('cv_var_'))) 
        else "tl_var_" + col.lower() 
        for col in trd7.columns
    ]
)

In [29]:
print(
    "Number of Modified CV Variables : " , len(
        [
            col 
            for col in trd8.columns 
            if col.startswith('cv_var_')
        ]
    ) , '\n' ,
    "Number of TL Variables : " , len(
        [
            col 
            for col in trd8.columns 
            if col.startswith('tl_var_')
        ]
    ) , '\n' ,
    "Total Number of Columns : " , len(trd8.columns) , '\n' ,
    "Total Number of Records : " , trd8.count() ,
    sep = ''
)

Number of Modified CV Variables : 2
Number of TL Variables : 2369
Total Number of Columns : 2371
Total Number of Records : 20


In [76]:
trd8.write.mode("overwrite").parquet(prep_data_path + "feature_tl_data.parquet")

25/05/08 09:43:25 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


In [ ]:
quit()